In [1]:
from pynq import Overlay
import time

In [2]:
ol = Overlay("design_1.bit")

In [3]:
mkl_gpio0 = ol.axi_gpio_0
mkl_gpio1 = ol.axi_gpio_1
mkl_div = ol.clk_wiz_0

In [4]:
XCLK_CLK_CFG_2 = 0x208
XCLK_CLK_CFG_23 = 0x25C

XGPIO_DATA0 = 0x0
XGPIO_DATA2 = 0x8

def cnfg0(QSPIio0):
    return 0

def cnfg1(QSPIio1):
    return 0

def cnfgdiv(ClkDiv):
    ClkDiv.write(XCLK_CLK_CFG_2, 0x40)                  #0x10 = 25 MHz, 0x40 = 6.25 MHz
    ClkDiv.write(XCLK_CLK_CFG_23, 0x03)
    return 0

def QSPI_Write32bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    mkl_gpio1.write(XGPIO_DATA0, 0x8000000E)            #start trigger, set nCS active low
    ##s = 0
    ##while s == 0:                                       #wait for start trigger done (bit 31 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    mkl_gpio1.write(XGPIO_DATA0, 0x0000000E)            #do no the 32bit transaction (release trigger)
    ##s = 0
    ##while s == 0:                                       #wait for shift done (bit 0 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
        
def QSPI_Write24bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    mkl_gpio1.write(XGPIO_DATA0, 0x8000001E)            #set bit for 24bit word to send
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    mkl_gpio1.write(XGPIO_DATA0, 0x0000001E)
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
        
def QSPI_TurnAround():
    mkl_gpio1.write(XGPIO_DATA0, 0x8000002E)            #set bit for 2bit turnaround
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    mkl_gpio1.write(XGPIO_DATA0, 0x0000002E)
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
    
def QSPI_Read32bit():
    mkl_gpio1.write(XGPIO_DATA0, 0x8000004E)            #set bit for 2bit turnaround
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    mkl_gpio1.write(XGPIO_DATA0, 0x0000004E)
    #s = 0
    #while s == 0:
    #    s = mkl_gpio1.read(XGPIO_DATA2)
        ##print(hex(s))
    s = mkl_gpio0.read(XGPIO_DATA2)
    return s
    
def QSPI_Deselect():
    mkl_gpio1.write(XGPIO_DATA0, 0x0000000F)            #release all nCS signals
    

In [5]:
cnfg0(mkl_gpio0)
cnfg1(mkl_gpio1)
cnfgdiv(mkl_div)

0

In [8]:
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345678)
QSPI_Write24bit(0xABCDEF00)      #is val << 8
QSPI_Write32bit(0x98765432)      #N-times
QSPI_Write32bit(0xACBD2013)
QSPI_Deselect()

In [9]:
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345678)
QSPI_Write24bit(0xABCDFE00)      #is val << 8
QSPI_TurnAround()
v1 = QSPI_Read32bit()            #N-times
v2 = QSPI_Read32bit()
QSPI_Deselect()
print(hex(v1))
print(hex(v2))

0x11111111
0x11111111
